In [21]:
using Pkg
Pkg.activate(".")

  Activating environment at `E:\PP\git\DataWrangler.jl\Project.toml`


In [22]:
using DataFrames, CSV, Dates, DataFramesMeta, Pipe

### Load in our Data

In [10]:
slave = CSV.read("data/Slave/Slave.csv", DataFrame)[:,3:end]

# Lets make the names lowercase for better readability
rename!(slave, lowercase.(names(slave)))

first(slave, 5)

,docdate,year,docno,notary,coder,dateinv,datesale,depot,parish
,Date,Float64,String?,String?,Float64?,Date?,Date?,Float64?,String?
1,1719-06-15,1719.0,missing,missing,1.0,missing,missing,55.0,OR
2,1719-06-15,1719.0,missing,missing,1.0,missing,missing,55.0,OR
3,1719-06-15,1719.0,missing,missing,1.0,missing,missing,55.0,OR
4,1719-06-15,1719.0,missing,missing,1.0,missing,missing,55.0,OR
5,1719-06-15,1719.0,missing,missing,1.0,missing,missing,55.0,OR


Make some type changes to data set.

In [14]:
slave.year = Int64.(slave.year)
()

()

Sale prices are listed in different currencies: 

| Currency of inventory inventory or sale value.| Equivalencies |
| ---                                           | :---          |
| p = piastre                                   | = 1 p         |
| s = peso                                      | = 1 p         |
| g = piastre gourde                            | = 1.75 p      |
| f = peso fuerte                               | = 1.75 p      |
| l = livre                                     | = 0.2 p       |
| t = livre tournois                            |  ?            |
| i = pound (livre) of indigo                   | = 1 p         |
| d = U.S. dollar                               | = 1 p         |
| z = unidentified                              |  ?            |

In [40]:
describe(slave)

,variable,mean,min,median,max,nunique,nmissing
,Symbol,Union…,Any,Union…,Any,Union…,Union…
1,docdate,,1719-06-15,,1820-12-31,11292,
2,year,1797.38,1719,1808.0,1820,,
3,docno,,0,,p.252,3298,9851
4,notary,,0ierre Pedasclaux,,vol. 22 Henry Hopkin,438,34036
5,coder,2.9168,1.0,3.0,9.0,,22998
6,dateinv,,0791-05-23,,1995-10-15,767,91966
7,datesale,,1404-03-31,,1999-10-30,7305,73671
8,depot,16.9609,0.0,0.0,58.0,,302
9,parish,,R,,sm,70,3864


In [40]:
@pipe slave |> 
  @where(_, .!ismissing.(:datesale) .& .!ismissing.(:dateinv)) |> 
  _[:, r"sale|inv|datesale|dateinv"] 

,dateinv,datesale,invcur,invvalue,invvalp,salecur,salevalu,salevalp
,Date?,Date?,String?,Float64?,Float64?,String?,Float64?,Float64?
1,1820-06-05,1820-06-20,D,500.0,500.0,D,2150.0,2150.0
2,1735-10-17,1735-10-18,missing,missing,missing,L,1200.0,240.0
3,1735-10-17,1735-10-18,missing,missing,missing,L,2695.0,539.0
4,1735-10-17,1735-10-18,missing,missing,missing,L,2695.0,539.0
5,1735-10-17,1735-10-18,missing,missing,missing,L,2695.0,539.0
6,1735-10-17,1735-10-18,missing,missing,missing,L,2060.0,412.0
7,1735-10-17,1735-10-18,missing,missing,missing,L,2060.0,412.0
8,1735-10-17,1735-10-18,missing,missing,missing,L,2650.0,530.0
9,1735-10-17,1735-10-18,missing,missing,missing,L,2650.0,530.0


In [82]:
notmissing(x) = !ismissing(x)

@linq slave |> 
    where(.!ismissing.(:invvalue) .& .!ismissing.(:salevalu) .& (:invvalue .!= 0) .& (:salevalu .!= 0) .&
    (:group .== 2)) |> 
    select(:invvalue, :salevalu, :year) |> 
    transform(salehigher = :salevalu .> :invvalue)

,invvalue,salevalu,year,salehigher
,Float64?,Float64?,Int64,Bool
1,500.0,960.0,1736,1
2,100.0,505.0,1736,1
3,300.0,590.0,1736,1
4,800.0,1070.0,1736,1
5,150.0,405.0,1736,1
6,1000.0,1250.0,1736,1
7,1000.0,2000.0,1736,1
8,1000.0,1090.0,1736,1
9,1000.0,1150.0,1737,1


In [11]:
[v in ["9", "Unnamed", "no name"] for v in slave.NAME ]

UndefVarError: UndefVarError: slave not defined

In [12]:
slave.NAME[[v in ["9", "Unnamed", "no name"] for v in slave.NAME ]]

UndefVarError: UndefVarError: slave not defined

In [13]:
sort(freqtable(slave.NAME), rev = true)

UndefVarError: UndefVarError: slave not defined

In [14]:
free  = CSV.read("data/Free/Free.csv")[:,3:end]

UndefVarError: UndefVarError: CSV not defined

In [15]:
free.YEAR = free.YEAR .|> Int

UndefVarError: UndefVarError: free not defined